# Main

## Import Libraries

In [1]:
# import library
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import trange

# sklearn
import sklearn
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.feature_selection import RFECV, SelectKBest, r_regression, f_regression
from sklearn.gaussian_process.kernels import Matern, RBF, CompoundKernel, Product, Sum, ExpSineSquared, RationalQuadratic
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, IsolationForest, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, make_scorer, f1_score, confusion_matrix
from sklearn.decomposition import PCA

# boost algorithm
import xgboost as xgb
import catboost as cat
import lightgbm as lgb

# torch
import torch
from torch import nn
from torch.nn import Module, Linear, Dropout
from torch.nn.functional import tanh, softmax, mse_loss, relu, sigmoid, binary_cross_entropy, nll_loss
from torch.optim import Adam, SGD

# bio library
import biosppy
from biosppy import storage
from biosppy.signals import ecg

# import ResNet
from ResNet import ResNet

DATA_DIR = "Data"
RESULT_DIR = "Result"

## Load Features Data

In [2]:
X_train_expert = np.load("./Data/X_train_features.npy")
X_train_DNN = np.load("./Data/X_train_DNN_features.npy")
y_train = np.load("./Data/y_train.npy")

X_test_expert = np.load("./Data/X_test_features.npy")
X_test_DNN = np.load("./Data/X_test_DNN_features.npy")

In [3]:
X_train_features = np.concatenate([X_train_expert, X_train_DNN], axis=1)
X_test_features = np.concatenate([X_test_expert, X_test_DNN], axis=1)

## Data Preprocessing

In [4]:
feature_select = SelectKBest(k=100)
feature_select.fit(X_train_features, y_train)
X_train_features = feature_select.transform(X_train_features)
X_test_features = feature_select.transform(X_test_features)

/Users/lanhongyi/opt/anaconda3/envs/aml_project/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [24] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/lanhongyi/opt/anaconda3/envs/aml_project/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


## Train and Test

In [5]:
# 预测数据
model = cat.CatBoostClassifier(iterations=2000, learning_rate=0.05, max_depth=5)
model.fit(X_train_features, y_train)
result = model.predict(X_test_features)

0:	learn: 1.2492932	total: 69.1ms	remaining: 2m 18s
1:	learn: 1.1348725	total: 79ms	remaining: 1m 18s
2:	learn: 1.0361830	total: 86.6ms	remaining: 57.7s
3:	learn: 0.9516290	total: 93.3ms	remaining: 46.5s
4:	learn: 0.8765514	total: 104ms	remaining: 41.7s
5:	learn: 0.8109858	total: 127ms	remaining: 42.2s
6:	learn: 0.7532731	total: 137ms	remaining: 38.9s
7:	learn: 0.7012270	total: 149ms	remaining: 37.1s
8:	learn: 0.6538479	total: 154ms	remaining: 34.1s
9:	learn: 0.6111828	total: 160ms	remaining: 31.8s
10:	learn: 0.5719789	total: 165ms	remaining: 29.9s
11:	learn: 0.5354077	total: 171ms	remaining: 28.3s
12:	learn: 0.5024088	total: 176ms	remaining: 26.8s
13:	learn: 0.4714037	total: 181ms	remaining: 25.6s
14:	learn: 0.4434456	total: 186ms	remaining: 24.6s
15:	learn: 0.4174575	total: 191ms	remaining: 23.7s
16:	learn: 0.3937955	total: 196ms	remaining: 22.9s
17:	learn: 0.3715530	total: 201ms	remaining: 22.1s
18:	learn: 0.3508232	total: 206ms	remaining: 21.5s
19:	learn: 0.3313756	total: 211ms	rem

In [6]:
result_df = pd.DataFrame(result.reshape((-1, 1)), columns=["y"]).reset_index()
result_df.columns = ["id", "y"]
result_df.to_csv("./Result/result_dnn.csv", index=False)